In [1]:
"""Utilities for constructing noisy GHZ states and checking positivity.
"""
from __future__ import annotations

from typing import Iterable, Sequence

import numpy as np
from qiskit.quantum_info import DensityMatrix, Statevector, Operator, SparsePauliOp
from quantum_utils import train_linear_svm_pauli, sample_random_pure_separable_state, classify_state_with_svm



def ghz3_noisy_state(p_noise: float) -> np.ndarray:
    """Return the 3-qubit noisy GHZ density matrix.

    Args:
        p_noise: Mixing probability with the maximally mixed state. Must satisfy 0 <= p_noise <= 1.

    Returns:
        An 8x8 numpy.ndarray representing the density matrix.
    """
    if not 0.0 <= p_noise <= 1.0:
        raise ValueError("p_noise must be between 0 and 1")

    ghz_state = (Statevector.from_label("000") + Statevector.from_label("111")) / np.sqrt(2)
    ghz_density = DensityMatrix(ghz_state).data

    identity = np.eye(8, dtype=complex) / 8.0
    return (1.0 - p_noise) * ghz_density + p_noise * identity


def partial_transpose(
    rho: np.ndarray,
    qubits: int | Iterable[int],
    num_qubits: int = 3,
) -> np.ndarray:
    """Return the partial transpose of ``rho`` over the selected qubits.

    Args:
        rho: Density matrix to transpose.
        qubits: Single qubit index or collection of indices to transpose.
        num_qubits: Total number of qubits in ``rho``.

    Returns:
        The partially transposed density matrix as a numpy array.
    """
    if isinstance(qubits, int):
        qubits = (qubits,)
    qubits = tuple(qubits)

    dims: Sequence[int] = (2,) * num_qubits
    rho_dm = DensityMatrix(rho, dims=dims)
    pt = rho_dm.partial_transpose(qubits)
    return pt.data


def is_positive_semidefinite(matrix: np.ndarray, atol: float = 1e-12) -> bool:
    """Check whether all eigenvalues of ``matrix`` are non-negative.

    Args:
        matrix: Matrix to test.
        atol: Absolute tolerance applied to eigenvalues (useful for numerical noise).

    Returns:
        ``True`` if every eigenvalue is >= -``atol``; ``False`` otherwise.
    """
    hermitian = 0.5 * (matrix + matrix.conj().T)
    eigenvalues = np.linalg.eigvalsh(hermitian)
    return bool(np.all(eigenvalues >= -atol))


__all__ = ["ghz3_noisy_state", "partial_transpose", "is_positive_semidefinite"]

In [2]:
def overlap_W(n):
    return (n-1)/n

def overlap_D(n):
    return n/(2*(n-1))

In [3]:
def GHZ(n):
    return DensityMatrix((Statevector.from_label("0"*n) + Statevector.from_label("1"*n)) / np.sqrt(2)).data

def EW_GHZ(n):
    dim = 2**n
    return 0.5 * np.eye(dim) - GHZ(n)


def cluster_state(n):
    from qiskit import QuantumCircuit
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.h(i)
    for i in range(n-1):
        qc.cz(i, i+1)
    state = Statevector.from_instruction(qc)
    return DensityMatrix(state).data

def EW_cluster(n):
    dim = 2**n
    cs = cluster_state(n)
    return 0.5 * np.eye(dim) - cs

def W_state(n):
    # build by adding Statevector objects explicitly
    svs = [Statevector.from_label('0'*i + '1' + '0'*(n - i - 1)) for i in range(n)]
    psi = svs[0]
    for s in svs[1:]:
        psi = psi + s
    psi = psi / np.sqrt(n)
    return DensityMatrix(psi).data

def EW_W(n):
    dim = 2**n
    w = (n - 1) / n * np.eye(dim) - W_state(n)  
    return w

def Dn2_n(n):
    #create (n/2,n) symmmtric Dicke state
    from itertools import combinations
    from qiskit import QuantumCircuit
    qc = QuantumCircuit(n)
    for i in combinations(range(n), n//2):
        for j in i:
            qc.x(j)
        qc.h(range(n))
        for j in i:
            qc.x(j)
    state = Statevector.from_instruction(qc)
    state /= np.sqrt(len(list(combinations(range(n), n//2))))
    return DensityMatrix(state).data

def EW_Dn2_n(n):
    dim = 2**n
    dn2 = Dn2_n(n)
    w = (n / (2*n - 2)) * np.eye(dim) - dn2
    return w

def max_min(EW,n):
    ew = EW(n)
    tr = []
    for i in range(1000):
        sep_state = sample_random_pure_separable_state(n)[0]
        tr.append(np.trace(ew @ sep_state))
    tr.sort()
    return tr[0], tr[-1]



In [25]:
a = ghz3_noisy_state(0.75)
print(a)

[[0.21875+0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.125  +0.j]
 [0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.09375+0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.09375+0.j 0.     +0.j]
 [0.125  +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.21875+0.j]]


In [26]:
a_t1 = partial_transpose(a, qubits=1)
print(a_t1)

[[0.21875+0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j 0.125  +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.09375+0.j 0.     +0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.125  +0.j 0.     +0.j 0.     +0.j 0.09375+0.j
  0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.09375+0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
  0.     +0.j 0.21875+0.j]]


In [27]:
print(is_positive_semidefinite(a_t1))

False


In [7]:
for i in range(3,8):
    print(f"n={i}, 'tr sep: ', 0.5, 0.5, overlap_W={overlap_W(i)}, overlap_D={overlap_D(i)}")
    print(f"n = {i}, 'tr_evol: ', -0.5, -0.5, overlap_W = {overlap_W(i) -1 }, overlap_D(i) = {overlap_D(i) - 1}")

n=3, 'tr sep: ', 0.5, 0.5, overlap_W=0.6666666666666666, overlap_D=0.75
n = 3, 'tr_evol: ', -0.5, -0.5, overlap_W = -0.33333333333333337, overlap_D(i) = -0.25
n=4, 'tr sep: ', 0.5, 0.5, overlap_W=0.75, overlap_D=0.6666666666666666
n = 4, 'tr_evol: ', -0.5, -0.5, overlap_W = -0.25, overlap_D(i) = -0.33333333333333337
n=5, 'tr sep: ', 0.5, 0.5, overlap_W=0.8, overlap_D=0.625
n = 5, 'tr_evol: ', -0.5, -0.5, overlap_W = -0.19999999999999996, overlap_D(i) = -0.375
n=6, 'tr sep: ', 0.5, 0.5, overlap_W=0.8333333333333334, overlap_D=0.6
n = 6, 'tr_evol: ', -0.5, -0.5, overlap_W = -0.16666666666666663, overlap_D(i) = -0.4
n=7, 'tr sep: ', 0.5, 0.5, overlap_W=0.8571428571428571, overlap_D=0.5833333333333334
n = 7, 'tr_evol: ', -0.5, -0.5, overlap_W = -0.1428571428571429, overlap_D(i) = -0.41666666666666663


In [13]:
for i in range(3,8):
    sep = []
    for j in range(1000):
        sep.append(sample_random_pure_separable_state(i)[0])
    for k in [EW_GHZ, EW_cluster, EW_W, EW_Dn2_n]:
            tr = [np.trace(k(i) @ a ) for a in sep]
            tr.sort()
            print(f'n={i}, EW={k.__name__}, min_sep={tr[0]}')
            print(f'n={i}, EW={k.__name__}, max_sep={tr[-1]}')

    

n=3, EW=EW_GHZ, min_sep=(0.052899909427762924-6.938893903907228e-18j)
n=3, EW=EW_GHZ, max_sep=(0.4999716775159967+5.312590645178972e-18j)
n=3, EW=EW_cluster, min_sep=(0.030947143970427072+0j)
n=3, EW=EW_cluster, max_sep=(0.49999963847677215-1.123504501238104e-17j)
n=3, EW=EW_W, min_sep=(0.13323493966398023+0j)
n=3, EW=EW_W, max_sep=(0.6664170314778317+1.0408340855860843e-17j)
n=3, EW=EW_W, min_sep=(0.13323493966398023+0j)
n=3, EW=EW_W, max_sep=(0.6664170314778317+1.0408340855860843e-17j)
n=3, EW=EW_Dn2_n, min_sep=(0.4480421756134482-1.474514954580286e-17j)
n=3, EW=EW_Dn2_n, max_sep=(0.7499926794628252+1.1140177322288558e-17j)
n=4, EW=EW_GHZ, min_sep=(0.14008940723472635-6.5052130349130266e-18j)
n=4, EW=EW_GHZ, max_sep=(0.4999658148453282-1.146543797403421e-17j)
n=3, EW=EW_Dn2_n, min_sep=(0.4480421756134482-1.474514954580286e-17j)
n=3, EW=EW_Dn2_n, max_sep=(0.7499926794628252+1.1140177322288558e-17j)
n=4, EW=EW_GHZ, min_sep=(0.14008940723472635-6.5052130349130266e-18j)
n=4, EW=EW_GHZ, m

In [ ]:
# Compute separable-state overlaps for GHZ, Cluster, W and D and show a table
import pandas as pd
from IPython.display import display

models = [('GHZ', EW_GHZ), ('Cl', EW_cluster), ('W', EW_W), ('D', EW_Dn2_n)]
rows = []
num_samples = 10000  # adjust for speed/accuracy

for n in range(3, 8):
    for label, func in models:
        EW = func(n)
        vals = []
        for _ in range(num_samples):
            rho_sep = sample_random_pure_separable_state(n)[0]
            vals.append(np.real(np.trace(EW @ rho_sep)))
        # theoretical overlaps when available
        if label == 'W':
            theo = overlap_W(n)
        elif label == 'D':
            theo = overlap_D(n)
        elif label == 'GHZ':
            theo = 0.5
        elif label == 'Cl':
            theo = 0.5
        else:
            theo = np.nan
        rows.append({
            'n': n,
            'model': label,
            'min_sep': float(np.min(vals)),
            'max_sep': float(np.max(vals)),
            'gap_sep': float(np.max(vals)-np.min(vals)),
            'diff_sep': float(theo - (np.max(vals)-np.min(vals))),
            'ratio': float(1-(np.max(vals)-np.min(vals))/theo),
            'theo_overlap': float(theo)
        })



In [26]:
df = pd.DataFrame(rows)
print('Separable-sampling summary (each row = model,n):')
display(df)
print('\nPivot table: min and max separable-trace by model')
pivot = df.pivot(index='n', columns='model', values=['min_sep','max_sep'])
display(pivot)

# Save to Excel (.xlsx) with num_samples and model names in filename
import glob, os, time
models_str = '_'.join([m[0] for m in models])
# look for any existing file with the same num_samples (regardless of model name suffix)
pattern = f"separable_summary_nsamples_{num_samples}_models_*.xlsx"
matches = glob.glob(pattern)

if matches:
    existing = matches[0]
    print(f'Found existing file for same sample count: {existing} — appending a new sheet')
    try:
        from openpyxl import load_workbook
        wb = load_workbook(existing)
        # choose next run index as sheet name
        run_idx = 1
        while f'run_{run_idx}' in wb.sheetnames:
            run_idx += 1
        sheet_name = f'run_{run_idx}'
        with pd.ExcelWriter(existing, engine='openpyxl', mode='a') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            # also save pivot to a companion sheet
            try:
                pivot.to_excel(writer, sheet_name=sheet_name + '_pivot')
            except Exception:
                pass
        print(f'Appended sheet "{sheet_name}" to {existing}')
    except Exception as e:
        # Fallback: append using a timestamped filename (if openpyxl not available or append fails)
        stamp = time.strftime('%Y%m%d_%H%M%S')
        fallback_sheet = f'run_{stamp}'
        try:
            with pd.ExcelWriter(existing, engine='openpyxl', mode='a') as writer:
                df.to_excel(writer, sheet_name=fallback_sheet, index=False)
                pivot.to_excel(writer, sheet_name=fallback_sheet + '_pivot')
            print(f'Appended sheet "{fallback_sheet}" to {existing} (fallback)')
        except Exception as e2:
            # As a last resort, create a new file with timestamp to avoid overwriting
            filename = f"separable_summary_nsamples_{num_samples}_models_{models_str}_{stamp}.xlsx"
            df.to_excel(filename, index=False)
            print(f'Could not append to {existing} — wrote new file {filename}\nError: {e2}')
else:
    # No existing file with same sample count: create a new file
    filename = f"separable_summary_nsamples_{num_samples}_models_{models_str}.xlsx"
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='summary', index=False)
            pivot.to_excel(writer, sheet_name='pivot')
        print(f'Saved summary to {filename}')
    except Exception as e:
        # Fallback to simple save (may still work if openpyxl missing)
        df.to_excel(filename, index=False)
        print(f'Saved summary to {filename} (fallback write). Error info: {e}')

Separable-sampling summary (each row = model,n):


,n,model,min_sep,max_sep,gap_sep,diff_sep,ratio,theo_overlap
0,3,GHZ,0.023208,0.499993,0.476785,0.023215,0.046431,0.500000
1,3,Cl,0.036788,0.499998,0.463210,0.036790,0.073580,0.500000
2,3,W,0.031807,0.666644,0.634837,0.031829,0.047744,0.666667
3,3,D,0.468129,0.749999,0.281870,0.468130,0.624174,0.750000
4,4,GHZ,0.085386,0.499996,0.414610,0.085390,0.170779,0.500000
5,4,Cl,0.093689,0.499998,0.406309,0.093691,0.187383,0.500000
6,4,W,0.359751,0.749998,0.390247,0.359753,0.479671,0.750000
7,4,D,0.548472,0.666667,0.118194,0.548473,0.822709,0.666667
8,5,GHZ,0.256943,0.499999,0.243056,0.256944,0.513889,0.500000
9,5,Cl,0.222219,0.499992,0.277772,0.222228,0.444455,0.500000



Pivot table: min and max separable-trace by model


min_sep                                 max_sep                      \
model        Cl         D       GHZ         W        Cl         D       GHZ   
n                                                                             
3      0.036788  0.468129  0.023208  0.031807  0.499998  0.749999  0.499993   
4      0.093689  0.548472  0.085386  0.359751  0.499998  0.666667  0.499996   
5      0.222219  0.583822  0.256943  0.484954  0.499992  0.625000  0.499999   
6      0.323557  0.586623  0.309656  0.633768  0.499998  0.600000  0.499998   
7      0.369059  0.578908  0.397048  0.741446  0.500000  0.583333  0.500000   

                 
model         W  
n                
3      0.666644  
4      0.749998  
5      0.799998  
6      0.833333  
7      0.857142

Saved summary to separable_summary_nsamples_10000_models_GHZ_Cl_W_D.xlsx


In [4]:
ghz_pauli = SparsePauliOp.from_operator(GHZ(3))


In [5]:
print(ghz_pauli)

SparsePauliOp(['III', 'IZZ', 'XXX', 'XYY', 'YXY', 'YYX', 'ZIZ', 'ZZI'],
              coeffs=[ 0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,
  0.125+0.j,  0.125+0.j])


In [10]:
a = train_linear_svm_pauli(3,ghz_pauli, class_weights=(1,2) )

In [11]:
test = []
for i in range(1000):
    test.append(sample_random_pure_separable_state(3))
result = classify_state_with_svm(test, a)


In [12]:
print(result)

['separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable', 'separable'

In [28]:
print(SparsePauliOp.from_list([('III', 2**(-3) + 0.j)]))

SparsePauliOp(['III'],
              coeffs=[0.125+0.j])
